Forecast the CocaCola prices and Airlines Passengers data set. Prepare a document for each model explaining 
how many dummy variables you have created and RMSE value for each model. Finally which model you will use for 
Forecasting.


In [ ]:
#importing Libraries
import pandas as pd
import numpy as np
from numpy import sqrt

from pandas import Grouper
from pandas import DataFrame

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from pandas.plotting import lag_plot
import statsmodels.formula.api as smf


In [ ]:
#import dataset
from google.colab import files
uploaded=files.upload()

In [ ]:
#Read the dataset
cocacola=pd.read_csv('CocaCola_Sales_Rawdata.csv')

In [ ]:
cocacola

# EDA

In [ ]:
#find shape of dataset
cocacola.shape

In [ ]:
cocacola.info()

In [ ]:
#calculate statistics of dataset
cocacola.describe()

In [ ]:
#check duplicate values
cocacola[cocacola.duplicated()].shape #No duplicate values

# Visualization

In [ ]:
#create lineplot
cocacola.plot()

In [ ]:
#Import seaborn library
import seaborn as  sns
sns.boxplot(data =cocacola)

In [ ]:
#Create histogram
cocacola.hist()

In [ ]:
#Density plot
cocacola.plot(kind="kde")

In [ ]:
#represent the dataset in array
np.array(cocacola["Sales"])

In [ ]:
#Density plot with histogram
import seaborn as sns
sns.set_theme()
rk= sns.distplot(cocacola['Sales'],kde=True)

In [ ]:
#Lagplot
from pandas.plotting import lag_plot
lag_plot(cocacola['Sales'])

# Data Preprocessing

In [ ]:
cocacola.head()

In [ ]:
# check length of dataset
len(cocacola) 

In [ ]:
cocacola['quarter'] = 0 # quater column created
for i in range(42):
    p=cocacola['Quarter'][i]
    cocacola['quarter'][i]=p[0:2]

In [ ]:
cocacola

In [ ]:
cocacola['quarter'].value_counts()

# Create dummy variables

In [ ]:
df_dummies=pd.DataFrame(pd.get_dummies(cocacola['quarter']),columns=['Q1','Q2','Q3','Q4'])
cc=pd.concat([cocacola,df_dummies],axis= 1)

In [ ]:
cc

In [ ]:
cc['t'] = np.arange(1,43)
cc['t_squared'] = cc['t']**2
cc["Sales_log"] =np.log(cocacola['Sales'])

In [ ]:
cc.head() # four dummy variables created

In [ ]:
train =cc.head(32)
test =cc.tail(10)

In [ ]:
cocacola['Sales'].plot()

# Build the model

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Linear Model
linear_model =smf.ols("Sales~t",data =train).fit()
linear_pred = pd.Series(linear_model.predict(test['t']))
linear_rmse =np.sqrt(mean_squared_error(np.array(test['Sales']),np.array(linear_pred)))
linear_rmse

In [ ]:
#Quadratic Model
quad_model =smf.ols("Sales~t+t_squared",data=train).fit()
quad_pred = pd.Series(quad_model.predict(test[['t','t_squared']]))
quad_rmse =np.sqrt(mean_squared_error(np.array(test['Sales']),np.array(quad_pred)))
quad_rmse

In [ ]:
# Exponential model
exp_model  =smf.ols("Sales_log~t",data=train).fit()
exp_pred =pd.Series(exp_model.predict(test['t']))
exp_rmse =np.sqrt(mean_squared_error(np.array(test['Sales']),np.array(exp_pred)))
exp_rmse

In [ ]:
data = {"MODEL":pd.Series(["rmse_linear","rmse_exp","rmse_quad"]),"RMSE_Values":pd.Series([linear_rmse,exp_rmse,quad_rmse,])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(['RMSE_Values'])

# Using ARIMA model

In [ ]:
data = pd.read_csv("CocaCola_Sales_Rawdata.csv",header=0,index_col=0, parse_dates=True)
data.head()

In [ ]:
#separate out a validation dataset
split_point = len(data) - 7
dataset_cc, validation_cc = data[0:split_point], data[split_point:]
print('Dataset_cc %d, Validation_cc %d' % (len(dataset_cc), len(validation_cc)))

In [ ]:
dataset_cc.to_csv('dataset_cc.csv', header=False)
validation_cc.to_csv('validation_cc.csv', header=False)

In [ ]:
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from math import sqrt

train = read_csv('dataset_cc.csv', header=None, index_col=0, parse_dates=True, squeeze=True)

train

In [ ]:
X = train.values
X = X.astype('float32')
train_size = int(len(X) * 0.50)
train, test = X[0:train_size], X[train_size:]

# Model Validation

In [ ]:
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    yhat = history[-1]
    predictions.append(yhat)
# observation
    obs = test[i]
    history.append(obs)
    print('>Predicted=%.3f, Expected=%.3f' % (yhat, obs))
# report performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

In [ ]:
data = {"MODEL":pd.Series(["rmse_linear","rmse_exp","rmse_quad","RMSE_ARIMA"]),"RMSE_Values":pd.Series([linear_rmse,exp_rmse,quad_rmse,rmse])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(['RMSE_Values'])

# Hence, we can observe ARIMA Model give least RMSE value.so ARIMA Model will be final model